insert data
* insert
  * create
* insert_one
* insert_many
* save

get data
* get
* find
  * to_list
  * first_or_none
* find_one
* find_many
* find_all

sync


when we would like to create application 

we want to create for users

that why our first class will be user

pydantic + beanie = ❤️


In [69]:
from pydantic import BaseModel


class User(BaseModel):
    name: str
    surname: str
    email: str

type(User)

pydantic._internal._model_construction.ModelMetaclass

In [152]:
not_hot_adam = User(name="Adam", surname="Brzyzek", email="nothotadam@gmail.com")
not_hot_adam

User(id=None, revision_id=None, name='Adam', surname='Brzyzek', email='nothotadam@gmail.com')

In [71]:
not_hot_adam.model_dump_json()

'{"name":"Adam","surname":"Brzyzek","email":"nothotadam@gmail.com"}'

In [72]:
not_hot_adam.model_dump()

{'name': 'Adam', 'surname': 'Brzyzek', 'email': 'nothotadam@gmail.com'}

In [73]:
from beanie import Document


class User(Document):
    name: str
    surname: str
    email: str

type(User)

pydantic._internal._model_construction.ModelMetaclass

if you run it you will get 'CollectionWasNotInitialized'

first initialize Collection

In [74]:
import os

from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient


client = AsyncIOMotorClient(os.getenv("MONGODB_URI"))
await init_beanie(
    database=client.workshop,
    document_models=[User],
    multiprocessing_mode=True,
)
client.drop_database(name_or_database=client.workshop)


<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/coolig/.pyenv/versions/3.11.9/lib/python3.11/asyncio/futures.py:387]>

In [75]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
hot_adam


User(id=None, revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [76]:
hot_adam.model_dump()

{'id': None,
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [77]:
User.insert(hot_adam)

<coroutine object Document.insert at 0x10e04c6a0>

coroutine

In [78]:
hot_adam.model_dump()

{'id': None,
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [79]:
await User.insert(hot_adam)

User(id=ObjectId('66c37bfdd4bd4b7536884de8'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [80]:
hot_adam.model_dump()


{'id': '66c37bfdd4bd4b7536884de8',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [81]:
client.drop_database(name_or_database=client.workshop)


<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/coolig/.pyenv/versions/3.11.9/lib/python3.11/asyncio/futures.py:387]>

In [82]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
hot_adam
await User.insert_one(hot_adam)


User(id=ObjectId('66c37bfdd4bd4b7536884de9'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

In [134]:
hot_adam.model_dump()

{'id': '66c37c25d4bd4b7536884dea',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [136]:
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com")
await User.create(hot_adam)
hot_adam.model_dump()

{'id': '66c37fd6d4bd4b7536884dec',
 'name': 'Adam',
 'surname': 'Brzyzek',
 'email': 'hotadam@gmail.com'}

In [137]:
adam = await User.get(hot_adam.id)
adam, type(adam)

(User(id=ObjectId('66c37fd6d4bd4b7536884dec'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 __main__.User)

In [87]:
await User.find_one(User.id == hot_adam.id)

User(id=ObjectId('66c37c25d4bd4b7536884dea'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')

find more powerfull

In [97]:
result = User.find()
print(result, type(result))

<beanie.odm.queries.find.FindMany object at 0x10ec80790> <class 'beanie.odm.queries.find.FindMany'>


In [99]:
result = await User.find().to_list()
print(result)
print(len(result))


[User(id=ObjectId('66c37bfdd4bd4b7536884de9'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'), User(id=ObjectId('66c37c25d4bd4b7536884dea'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]
2


[User(id=ObjectId('66c37bfdd4bd4b7536884de9'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37c25d4bd4b7536884dea'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37fc5d4bd4b7536884deb'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37fd6d4bd4b7536884dec'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [148]:
result = await User.find().first_or_none()
print(result)
print(type(result))

id=ObjectId('66c37bfdd4bd4b7536884de9') revision_id=None name='Adam' surname='Brzyzek' email='hotadam@gmail.com'
<class '__main__.User'>


pymongo syntax - ble string XD

In [149]:
await User.find({"_id": hot_adam.id}).to_list()


[User(id=ObjectId('66c37fd6d4bd4b7536884dec'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [154]:
await User.find_many().to_list()


[User(id=ObjectId('66c37bfdd4bd4b7536884de9'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37c25d4bd4b7536884dea'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37fc5d4bd4b7536884deb'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37fd6d4bd4b7536884dec'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]

In [153]:
await User.find_all().to_list()


[User(id=ObjectId('66c37bfdd4bd4b7536884de9'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37c25d4bd4b7536884dea'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37fc5d4bd4b7536884deb'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com'),
 User(id=ObjectId('66c37fd6d4bd4b7536884dec'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com')]